In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import language_modelling_seq2seq

In [2]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(language_modelling_seq2seq)    
net = language_modelling_seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, encoder_layers = 2, 
                                            decoder_hidden_units = 100, decoder_layers = 2)
net.to(device)
net.fit(source_1, target_1, epochs = 1, batch_size = 150, lr = 0.0001, verbose = 3)

Net parameters: 408,795



Epoch:    1, Loss: 2.6396
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e  ee   e', ' e  e   e ', '   e   e  ', '  e   e   ', ' e   e    ']
greedy_search
['          ', '          ', '          ', '          ', '          ']
tensor([-12.3500, -12.1700, -12.1300, -10.3300,  -9.6600], device='cuda:0')
sample
['d btwmm nt', ' arle usah', 'e\naoos,wii', ' t ie\nqobP', ' nwitakkce']
beam_search
torch.Size([25, 10])
torch.Size([25, 2, 100])
[['          ', '          ', '          ', '          ', '          '],
 ['         \n', '         \n', '         \n', '         K', '         K'],
 ['        \n ', '        \n ', '        \n ', '         \n', '         \n'],
 ['         K', '         K', '       \n  ', '        \n ', '         .'],
 ['         .', '         .', '         K', '       \n  ', '        \n ']]
tensor([[-12.3500, -12.1700, -12.1300, -10.3300,  -9.6600],
        [-14.6500, -14.5100, -14.5100, -12.8000, -12.0600],
        [-14.6800, -14.54

In [9]:
net.cpu().beam_search(source_1[:10000].cpu(), verbose = 1, batch_size = 50)

torch.Size([50000, 10])
torch.Size([50000, 2, 100])


(tensor([[[ 1,  5,  5,  ...,  5,  5,  5],
          [ 1,  5,  5,  ...,  5, 19,  5],
          [ 1,  5,  5,  ..., 19,  5,  5],
          [ 1,  5,  5,  ...,  5,  5,  4],
          [ 1,  5,  5,  ...,  5,  5, 19]],
 
         [[ 1,  5,  5,  ...,  5,  5,  5],
          [ 1,  5,  5,  ...,  5, 19,  5],
          [ 1,  5,  5,  ...,  5,  5,  4],
          [ 1,  5,  5,  ..., 19,  5,  5],
          [ 1,  5,  5,  ...,  5,  5, 19]],
 
         [[ 1,  5,  5,  ...,  5,  5,  5],
          [ 1,  5,  5,  ...,  5, 19,  5],
          [ 1,  5,  5,  ...,  5,  5,  4],
          [ 1,  5,  5,  ..., 19,  5,  5],
          [ 1,  5,  5,  ...,  5,  4,  5]],
 
         ...,
 
         [[ 1,  5,  5,  ...,  5,  5,  5],
          [ 1, 71,  5,  ...,  5,  5,  5],
          [ 1,  5,  5,  ..., 19,  5,  5],
          [ 1,  5,  5,  ...,  5,  5, 48],
          [ 1,  5, 71,  ...,  5,  5,  5]],
 
         [[ 1,  5,  5,  ...,  5,  5,  5],
          [ 1, 71,  5,  ...,  5,  5,  5],
          [ 1,  5,  5,  ..., 19,  5,  5],
      